In [ ]:
!pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install numpy==1.18.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 39.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: Could not build wheels for numpy, which is required to install pyproject.toml-based projects


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pprint
import pandas as pd
import tempfile
from typing import Dict, Text
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [ ]:
class Config: 
    
    embedding_dimension = 32
    
    model_path = "model.tf"
    
config = Config()

In [ ]:
# Ratings data.
df = pd.read_csv('/content/data.csv')
#Skills
skill = pd.read_csv('/content/data3.csv')

In [ ]:
df["skills"] = df['skill_1'].astype(str) +", "+ df['skill_2'].astype(str) +", "+ df['skill_3'].astype(str) +", "+ df['skill_4'].astype(str)

In [ ]:
user = df[['id', 'name', 'skills', 'job_interest']]
user

,id,name,skills,job_interest
0,AAJ603,Chris Westphalen,"PHP, Visual Design, Google Cloud, Flutter Deve...",.NET Developer
1,HNA156,Koral Nilges,"Node.js, Information Architecture, Salesforce ...",Advance Data Analyst
2,FND122,Biron Mavin,"C#, Wireframing, Google Cloud, Flutter Develop...",AI Researcher
3,FMD294,Rurik Iwanowicz,"SQL, Usability Testing, DigitalOcean, Android ...",android architect
4,XVU891,Bobbee Foord,"Node.js, Wireframing, AWS, Flutter Development",Android Developer
...,...,...,...,...
495,JCH211,Hendrika Amoss,"React, Decision Trees, Xamarin, Digital Ocean",Web Designer
496,KWK881,Sile Watsham,"Django, Classification, Java, Google Cloud",Web Developer
497,BUX671,Dolf Gavin,"HTML/CSS, Random Forest, Ionic, Alibaba Cloud",Web Service Integration Specialist
498,EYD719,Linnea Houndsom,"Angular, Naive Bayes, Ionic, VMware Cloud",Windows Application Developer


In [ ]:
user = tf.data.Dataset.from_tensor_slices(dict(user))
skill = tf.data.Dataset.from_tensor_slices(dict(skill))

In [ ]:
# get the first rows of the movies dataset
for m in skill.take(5):
  print(m)

{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'yn1'>, 'skill': <tf.Tensor: shape=(), dtype=string, numpy=b'Communication'>}
{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'yn2'>, 'skill': <tf.Tensor: shape=(), dtype=string, numpy=b'Customer Service'>}
{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'yn3'>, 'skill': <tf.Tensor: shape=(), dtype=string, numpy=b'Financial Analysis'>}
{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'yn4'>, 'skill': <tf.Tensor: shape=(), dtype=string, numpy=b'Leadership'>}
{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'yn5'>, 'skill': <tf.Tensor: shape=(), dtype=string, numpy=b'Marketing'>}


In [ ]:
user = user.map(lambda x: {
    "id": x["id"],
    "name": x["name"],
    "skills": x["skills"],
    "job_interest": x["job_interest"]
})
skill = skill.map(lambda x: {
    "id": x["id"],
    "skill": x["skill"]
}) 

In [ ]:
user_array = []
skill_array = []
for data in user:
    user_array.append(data['id'].numpy().decode())
for data in skill:
    skill_array.append(data['skill'].numpy().decode())


#skills_array = tf.squeeze(skills).numpy().astype(str).tolist()
#jobs_array = tf.squeeze(jobs).numpy().astype(str).tolist()

user_ids_vocabulary = tf.keras.layers.StringLookup()
user_ids_vocabulary.adapt(user_array)

skill_titles_vocabulary = tf.keras.layers.StringLookup()
skill_titles_vocabulary.adapt(skill_array)


In [ ]:
for data in skill:
  print(data['skill'])

tf.Tensor(b'Communication', shape=(), dtype=string)
tf.Tensor(b'Customer Service', shape=(), dtype=string)
tf.Tensor(b'Financial Analysis', shape=(), dtype=string)
tf.Tensor(b'Leadership', shape=(), dtype=string)
tf.Tensor(b'Marketing', shape=(), dtype=string)
tf.Tensor(b'Negotiation', shape=(), dtype=string)
tf.Tensor(b'Project Management', shape=(), dtype=string)
tf.Tensor(b'Sales', shape=(), dtype=string)
tf.Tensor(b'Strategic Planning', shape=(), dtype=string)
tf.Tensor(b'Time Management', shape=(), dtype=string)
tf.Tensor(b'application security', shape=(), dtype=string)
tf.Tensor(b'cloud security', shape=(), dtype=string)
tf.Tensor(b'compliance', shape=(), dtype=string)
tf.Tensor(b'cybersecurity awareness', shape=(), dtype=string)
tf.Tensor(b'data privacy', shape=(), dtype=string)
tf.Tensor(b'identity and access management', shape=(), dtype=string)
tf.Tensor(b'incident response', shape=(), dtype=string)
tf.Tensor(b'malware analysis', shape=(), dtype=string)
tf.Tensor(b'network sec

In [ ]:
class JobLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.
 
  def __init__(
      self,
      user_model: tf.keras.Model,
      skill_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()
 
    # Set up user and movie representations.
    self.user_model = user_model
    self.skill_model = skill_model

 
    # Set up a retrieval task.
    self.task = task
 
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.
 
    user_embeddings = self.user_model(features["id"])
    skill_embeddings = self.skill_model(features["skill"])

 
    return self.task(user_embeddings, skill_embeddings)

In [ ]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    tf.keras.Input(shape=[], dtype=tf.string, name='id'),
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
skill_model = tf.keras.Sequential([
    tf.keras.Input(shape=[], dtype=tf.string, name='skill'),
    skill_titles_vocabulary,
    tf.keras.layers.Embedding(skill_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        skill.batch(128).map(skill_model)
    )
)

/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['id'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


In [ ]:
# Create a retrieval model.
model = JobLensModel(user_model, skill_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
 
# Train for 3 epochs.
model.fit(skill.batch(4096), epochs=50)

Epoch 1/50
1/1 [==============================] - 2s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0100 - factorized_top_k/top_5_categorical_accuracy: 0.0700 - factorized_top_k/top_10_categorical_accuracy: 0.1100 - factorized_top_k/top_50_categorical_accuracy: 0.5000 - factorized_top_k/top_100_categorical_accuracy: 0.9900 - loss: 460.5175 - regularization_loss: 0.0000e+00 - total_loss: 460.5175
Epoch 2/50
1/1 [==============================] - 0s 75ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0100 - factorized_top_k/top_5_categorical_accuracy: 0.0600 - factorized_top_k/top_10_categorical_accuracy: 0.1000 - factorized_top_k/top_50_categorical_accuracy: 0.4900 - factorized_top_k/top_100_categorical_accuracy: 0.9900 - loss: 460.5174 - regularization_loss: 0.0000e+00 - total_loss: 460.5174
Epoch 3/50
1/1 [==============================] - 0s 67ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0200 - factorized_top_k/top_5_categorical_accuracy: 0.0600 - factori

In [ ]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.skill_model)
index.index_from_dataset(
    user.batch(100).map(lambda x: (x['id'], model.user_model(x['id'])))
)

# Get some recommendations.
ids = tf.constant(["yn45"])
_, titles = index(tf.constant(ids))
for i in range(len(ids)):
    print(f"Recommendations for user id {ids[i].numpy()}: {titles[i].numpy()}")

Recommendations for user id b'yn45': [b'JJY397' b'JFZ010' b'ADJ404' b'HKX026' b'GPS072' b'IHO320' b'DKX403'
 b'FGA047' b'DUC127' b'XLE736']


In [ ]:
def recommend(id):
    # Use brute-force search to set up retrieval using the trained representations.
    index = tfrs.layers.factorized_top_k.BruteForce(model.skill_model)
    index.index_from_dataset(
        user.batch(100).map(lambda x: (x['id'], model.user_model(x['id'])))
    )
    
    # Get recommendations for the given user ID.
    ids = tf.constant([id])
    _, titles = index(tf.constant(ids))
    
    # Print the recommendations.
    recommendations = titles.numpy().tolist()
    return recommendations
  

In [ ]:
recommend('yn45')

[[b'JJY397',
  b'JFZ010',
  b'ADJ404',
  b'HKX026',
  b'GPS072',
  b'IHO320',
  b'DKX403',
  b'FGA047',
  b'DUC127',
  b'XLE736']]